In [14]:
### Library Import Initialization

import numpy as np
import math
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")

In [15]:
### Function to Import Stock Tickers and Calculate Final Stock Price

def import_stock_data(tickers, start_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date)['Adj Close']
    return data

tickers = 'GOOG'
stock_data = import_stock_data(tickers, '2018-01-01')
# Get the Current Stock Price (Starting Node of Tree)
S_0 = stock_data[tickers].iloc[-1]
S_0

[*********************100%%**********************]  1 of 1 completed


153.94000244140625

In [16]:
### Sigma Calculation 
def compute_sigma(data):
    # Compute the standard deviation of returns
    sigma = np.std(data) / 100
    return sigma

get_sigma = compute_sigma(stock_data)
sigma = get_sigma.values[0]
sigma


0.32998271375161553

In [17]:
### Function to Compute the Binomial Expected Value 

def american_binomial_model(S_0, K, T, r, sigma, option_type = 'call', n = 1000):
    """
    Parameters:
    S_0 (float): Current stock price (starting node of tree)
    K (float): Strike price of the option
    T (float): Time to expiration (in years)
    r (float): Risk-free interest rate
    sigma (float): Volatility of the underlying stock
    option_type (str): Type of option ('call' or 'put')
    n (int): Number of steps in the binomial tree
    Returns:
    float: Option price
    """
    # Compute Delta T
    dt = T / n
    # Compute up and down parameters
    u = np.exp(sigma * np.sqrt(dt))
    d = np.exp(-(sigma * np.sqrt(dt)))
    # Compute probability of Increase (Probability of a decrease is 1 - p)
    p = (np.exp(r * dt) - d) / (u - d)

    # Init arrays for strike prices and option values
    S_n = np.zeros((n + 1, n + 1))
    option_values = np.zeros((n + 1, n + 1))

    # Generate the stock prices at each node S_n (points on Binomial Tree)
    ''' 
    The expression essentially computes the stock price at node n by starting from the initial price S_0​ and multiplying it by the 
    factors u and d corresponding to the number of up and down movements, respectively, needed to reach node n.
    S_n​ = S_0 * u^m * d^(n−m)
    m (int): Number of up movements from the initial node to node n, calculated as m = n - i
    '''
    for j in range(n+1):
        for i in range(n+1):
            S_n[i, j] = S_0 * (u ** (j - i)) * (d ** i)

    # Compute the option value (intrinsic value) at the final node
    for i in range(n+1):
        if option_type == 'call':
            option_values[i, n] = max((S_n[i, n] - K), 0)

        if option_type == 'put':    
            option_values[i] = max((K - S_n[i, n]), 0)
    #return option_values

    # Recursively iterate through the tree backwards to compute the option values at previous nodes
    # Outer loop iterates over the time steps of the binomial tree in reverse order, starting from the last time step (n) and ending at the first time step (1)
    for j in range(n - 1, -1, -1):
    # Inner loop iterates over each node at each time step
        for i in range(j + 1):
            option_values[i, j] = np.exp(-r * dt) * (p * option_values[i, j + 1] + (1-p) * option_values[i + 1, j + 1])
    
    # Return the option value at the initial node
    option_value = option_values[0, 0]

    # Calculate uptick and downtick values
    up_factor = u - 1
    down_factor = 1 - d

    return option_value, up_factor, down_factor

### Function Output
option_type = 'put'
option_val, up_factor, down_factor = american_binomial_model(S_0, K = 139, T = 1, r = 0.05, sigma = sigma, option_type = option_type, n = 100)
print('The value of the ' + option_type + ' option price is: ' + str(round(option_val, 4)))
print('Up Factor: ' + str(round(up_factor, 4)))
print('Down Factor: ' + str(round(down_factor, 4)))


The value of the put option price is: 9.8656
Up Factor: 0.0335
Down Factor: 0.0325


In [18]:
### Manual Test (Accuracy Check)
option_test, up_factor, down_factor = american_binomial_model(S_0 = 100, K = 95, T = 0.5, r = 0.08, sigma = 0.3, option_type = 'put', n = 5)
print('The value of the ' + option_type + ' option price is: ' + str(round(option_test, 4)) + ', and the value is supposed to be: 4.6277')
print('Up Factor: ' + str(round(up_factor, 4)))
print('Down Factor: ' + str(round(down_factor, 4)))

The value of the put option price is: 4.6277, and the value is supposed to be: 4.6277
Up Factor: 0.0995
Down Factor: 0.0905
